In [1]:
import numpy as np

Esta función utiliza el algoritmo Levinson-Durvin para determinar los coeficientes de un modelo AR (autoregresivo) de orden p para la señal "data", una predicción de la señal dada a partir de sus últimas p muestras y los residuos de la predicción. Si se define lev = levinson(data, p) entonces lev[0] serán los coeficientes del modelo, lev[1] será la predicción lineal de la señal y lev[2] serán los residuos

In [2]:
def levinson(data,p):
    N = len(data)
    r = np.correlate(data-np.mean(data),data-np.mean(data),'full')/(N*np.std(data)**2)
    R = np.flip(r[0:N])
    E = np.zeros(p+1); k = np.zeros(p+1); Coef = np.zeros((p+1,p+1)); E[0] = R[0]
    for m in np.arange(1,p+1):
        if m == 1:
            k[m] = - R[m]/E[m-1]
            Coef[m,1] = k[m]
            E[m] = (1-k[m]**2)*E[m-1]
        else:
            s = 0
            for i in np.arange(1,m):
                s = s + Coef[m-1,i]*R[m-i]
                Coef[m,i] = Coef[m-1,i] + Coef[m,m]
            k[m] = - (R[m] + s)/E[m-1]
            Coef[m,m] = k[m]
            for i in np.arange(1,m):
                Coef[m,i] = Coef[m-1,i] + Coef[m,m] * Coef[m-1,m-i]
            E[m] = (1-k[m]**2)*E[m-1]
    coef = Coef[p,:]
    dataEstim = np.zeros(N); dataEstim[0] = 0
    for i in np.arange(1,N):
        s = 0
        for j in np.arange(1,p):
            if i - j >= 0:
                s = s - coef[j] * data[i-j]
        dataEstim[i] = s
    Residuos = data - dataEstim
    return coef[1:p+1], dataEstim, Residuos